In [1]:
# -*- coding: utf-8 -*-
from hanspell import spell_checker
from UrlParserForKBO import UrlParserForKBO
from Parser_DaumKBO import Parser_DaumKBO
from Parser_KBO import Parser_KBO
from sklearn import tree
from sklearn.externals.six import StringIO
from TransformeTeamName import TransformeTeamName

import pandas as pd
import numpy as np
import MySQLdb
import random
import datetime
import os
import sys

In [2]:
# DecisionTree - 서두
path_Intro = './DecisionLearning//DecisionTree_sample.csv'
df_Intro = pd.read_csv(path_Intro,index_col=None,header=None,names=['1H','2H','3H','4H','5H','6H','7H','8H','9H','10H','11H','12H','sum','context'])
score_Intro = np.array(df_Intro.reindex(columns=['1H','2H','3H','4H','5H','6H','7H','8H','9H','10H','11H','12H','sum']))
context_Intro = np.array(df_Intro.get('context'))
contextSentence = [' ','승리하였다. ','접전끝에 승리를 하였다. ','역전에 성공했다. ','완승을 하였다. ','무승부로 끝이났다. ','영봉승을 하였다. ']
clf_Intro = tree.DecisionTreeClassifier(criterion='entropy')
clf_Intro = clf_Intro.fit(score_Intro,context_Intro)

In [51]:
def changeWithParam(sentence, *wordBox):
    '''
    Change '()'(bracket) to element of wordBox(List) in Random.
    return type : String
    '''
    for element in wordBox:
        print element
        word=element[random.randint(0,len(element)-1)]
        sentence=sentence.replace(u'()',word,1)
    return sentence

In [4]:
def getClassifier():
    return clf_Intro.predict([x-y for x,y in zip(Parser_KBO.h_score[:13],Parser_KBO.a_score[:13])])[0]

In [5]:
def writeDate(parser_KBO):    
    return Parser_KBO.boxScore['date']

In [6]:
def writeHead(parser_KBO,parser_DaumKBO,contextClassifier):
#     문장뭉치
    sentenceGroup=[
        'at ascore vs ht hscore'
    ]
    
#     문장뽑기
    Head=sentenceGroup[random.randint(0,len(sentenceGroup)-1)]
    
#     문장만들기
    Head=sentence
    Head=Head.replace('at',Parser_KBO.a_team)
    Head=Head.replace('ht',Parser_KBO.h_team)
    Head=Head.replace('hscore',str(Parser_KBO.h_score[12]))
    Head=Head.replace('ascore',str(Parser_KBO.a_score[12]))
    
    return Head

In [36]:
def writeIntro(Parser_KBO,Parser_DaumKBO,contextClassifier):
#     문장뭉치
    sentences1Gorup=[
        changeWithParam(u'winTeam_name는 date stadium에서 () loseTeam_name와의 ()에서 winTeam_sum-loseTeam_sum 으로 context',
                        [u'열린',u'열렸던',u'치뤄진'],
                        [u'승부',u'경기',u'대결']
                       ),
        changeWithParam(u'date stadium에서 () away_name와 home_name와의 ()에서 away_sum-home_sum으로 winTeam_name이 context',
                        [u'열린',u'열렸던',u'치뤄진'],
                        [u'승부',u'경기',u'대결']
                       ),
        changeWithParam(u'away_name와 home_name의 ()가 date stadium에서 있었던 오늘 away_name는 away_sum, home_name는 home_sum로 winTeam_name가 경기에서 context',
                        [u'승부',u'경기',u'대결']
                       ),
        changeWithParam(u'date stadium에서 있었던 away와 home과의 (), winTeam이 winScore loseTeam이 loseScore로 winTeam_name이 context',
                        [u'승부',u'경기',u'대결']
                       ),
        changeWithParam(u'홈팀 home_name과 어웨이팀 away_name ()경기가 오늘 date 펼쳐졌다. stadium경기장에서 () 이 경기에서 winTeam_name이 winTeam_sum : loseTeam_sum으로 loseTeam_name을 context',
                        [u'승부',u'경기',u'대결'],
                        [u'열린',u'열렸던',u'치뤄진']
                       )
        ]
                        
    sentences2Group=[
        changeWithParam(u'winTeam_name 선발투수로 나온 winTeam_StarterPitcher선수가 INN이닝동안 NP개를  던지고 H피안타 BB볼넷 SO탈삼진 R실점의 ()를 () ()',
#                         선발투수 승수가 5이상이면 완벽투구
                        [u'역투',u'호투',u'투구'] if Parser_DaumKBO['winTeam']['StarterPitcherWinCount']>5 else [u'완벽투구'], 
                        [u'펼치며',u'던지며'],
                        [u'승리를 리드했다.',u'승리를 이끌었다.',u'팀의 승리를 주도했다.'] if Parser_DaumKBO['winTeam']['StarterPitcherWinCount']>5 else ['승리에 기여했다.','경기를 해나갔다.']
                       )
#         changeWithParam(u'본 ()에서 winTeam_name winTeam_StarterPitcher선수가 투구수 피안타H 의 기록을 보여주며 호투를 하였다. ',
#                         [u'경기',u'승부',u'대결'])
#         changeWithParam(u'() ()에서 winTeam_name의  winTeam_StarterPitcher선수가 winTeam_Start이닝을 호투하며, R실점 SO삼진을 잡아내며 승리를 이끌었다. ',
#                        [u'이번',u'본'],[u'승부',u'경기',u'대결'],[''])
    ]
    
#     문장뽑기
    sentence1=sentences1Gorup[random.randint(0,len(sentences1Gorup)-1)]
    sentence2=sentences2Group[random.randint(0,len(sentences2Group)-1)]
    
#     문장만들기

    #------- BEGIN Intro1-------#
    sentence1=sentence1.replace('winTeam_name',Parser_KBO.boxScore['winTeam']['name'])
    sentence1=sentence1.replace('loseTeam_name',Parser_KBO.boxScore['loseTeam']['name'])
    sentence1=sentence1.replace('winTeam_sum',str(Parser_KBO.boxScore['winTeam']['sum']))
    sentence1=sentence1.replace('loseTeam_sum',str(Parser_KBO.boxScore['loseTeam']['sum']))
    sentence1=sentence1.replace('home_sum',str(Parser_KBO.boxScore['home']['sum']))
    sentence1=sentence1.replace('away_sum',str(Parser_KBO.boxScore['away']['sum']))
    sentence1=sentence1.replace('stadium',Parser_DaumKBO.stadium)
    sentence1=sentence1.replace('date',str(int(Parser_KBO.boxScore['date'][4:6]))+u'월'+str(int(Parser_KBO.date[6:8]))+u'일'+Parser_KBO.date[8:])
    sentence1=sentence1.replace('home',Parser_KBO.boxScore['home']['name'])
    sentence1=sentence1.replace('away',Parser_KBO.boxScore['away']['name'])    
    
    # Intro1 예외처리
    #'context' -> 무승부
    if Parser_KBO.boxScore['home']['sum']==Parser_KBO.boxScore['away']['sum']:
        sentence1=sentence1.replace('context',context_sen_Intro[5].decode('utf-8'))    
    #'context' -> 영봉승
    elif Parser_KBO.boxScore['home']['sum']==0 or Parser_KBO.boxScore['away']['sum']==0:
        sentence1=sentence1.replace('context',context_sen_Intro[6].decode('utf-8'))    
    #'context'-> 나머지의 경우
    else:
        sentence1=sentence1.replace('context',context_sen_Intro[contextClassifier].decode('utf-8'))
    #------- END Intro1-------#

    #------- BEGIN Intro2-------#
    sentence2=sentence2.replace('winTeam_name',Parser_KBO.boxScore['winTeam']['name'])
    sentence2=sentence2.replace('mvp_pitcher',Parser_KBO.boxScore['winTeam']['pitRecord'].ix[Parser_DaumKBO.StarterPitcher['winTeam']['StarterPitcher']][u'선수명'])
    sentence2=sentence2.replace('ERA',Parser_KBO.boxScore['winTeam']['pitRecord'].ix[Parser_DaumKBO.StarterPitcher['winTeam']['StarterPitcher']][u'평균자책점'])
    sentence2=sentence2.replace('INN',Parser_KBO.boxScore['winTeam']['pitRecord'].ix[Parser_DaumKBO.StarterPitcher['winTeam']['StarterPitcher']][u'이닝'])
    sentence2=sentence2.replace('BB',Parser_KBO.boxScore['winTeam']['pitRecord'].ix[Parser_DaumKBO.StarterPitcher['winTeam']['StarterPitcher']][u'4사구'])
    sentence2=sentence2.replace('BN',Parser_KBO.boxScore['winTeam']['pitRecord'].ix[Parser_DaumKBO.StarterPitcher['winTeam']['StarterPitcher']][u'타자'])
    sentence2=sentence2.replace('NP',Parser_KBO.boxScore['winTeam']['pitRecord'].ix[Parser_DaumKBO.StarterPitcher['winTeam']['StarterPitcher']][u'투구수'])
    sentence2=sentence2.replace('SO',Parser_KBO.boxScore['winTeam']['pitRecord'].ix[Parser_DaumKBO.StarterPitcher['winTeam']['StarterPitcher']][u'삼진'])            
    sentence2=sentence2.replace('ER',Parser_KBO.boxScore['winTeam']['pitRecord'].ix[Parser_DaumKBO.StarterPitcher['winTeam']['StarterPitcher']][u'자책'])
    sentence2=sentence2.replace('R',Parser_KBO.boxScore['winTeam']['pitRecord'].ix[Parser_DaumKBO.StarterPitcher['winTeam']['StarterPitcher']][u'실점'])
    sentence2=sentence2.replace('H',Parser_KBO.boxScore['winTeam']['pitRecord'].ix[Parser_DaumKBO.StarterPitcher['winTeam']['StarterPitcher']][u'피안타'])
    
    #-------END Intro2-------#
    return sentence1+sentence2

In [37]:
# 결론작성
def writeConc(Parser_KBO,Parser_DaumKBO,contextClassifier):
#     문장뭉치
    sentence1Group=[
    changeWithParam(u'() winTeam_name은 winTeam_win_lose_winCount승 winTeam_win_lose_drawCount무 winTeam_win_lose_loseCount패 를 기록하게 되었다.',
                    [u'이날의 승리로',u'오늘의 승리로',u'이로인해']),
    changeWithParam(u'() () away은 away_win_lose_winCount승 away_win_lose_drawCount무 away_win_lose_loseCount패,home은 home_win_lose_winCount승 home_win_lose_drawCount무 home_win_lose_loseCount패 를 기록하게 되었다.',
                    [u'이날의',u'오늘의',u'오늘 있었던',u'오늘 열렸던',u'오늘 치뤄진'],
                    [u'경기로',u'승부로',u'대결로'])
    ]
    sentence2Group=[
    u'',
    changeWithParam(u'() loseTeam_win_lose_winCount승 loseTeam_win_lose_drawCount무 loseTeam_win_lose_loseCount패 를 기록하게 되었다.',
#                     1:승리/2:접전/3:역전/4:완승 각 경우에 대한 다른 문장
                    [u'패배의 아픔을 맛본',u'패배한'] if contextClassifier==1 else 
                    ['u아쉽게 패배한',u'안타깝게 패배한',u'접전끝에 패배한'] if contextClassifier==2 else
                    [u'역전을 당한',u'역전을 허용한'] if contextClassifier==3 else
                    [u'완패한',u'큰 점수차로 패배한']
    )]
    
    
#     순위 변동,연승의 내용은 필수
    essentialSentenceGroup=[]
    #      순위가 올라간경우
    if parser_DaumKBO.rank['winTeam'][1]>0:
        essentialSentenceGroup.append(
            changeWithParam(u'winTeam_name은 리그winTeam_rank+1위에서 () winTeam_rank위로 () ',
                            [u'1순위 올라가',u'한 순위 올라간'],
                            [u'안착하게 되었다.',u'팀의 순위가 상승하였다.']
                           )
            )
    #      순위가 내려간 경우
    if parser_DaumKBO.rank['loseTeam'][1]<0:
        essentialSentenceGroup.append(
            changeWithParam(u'한편, loseTeam_name은 리그 loseTeam_rank-1위에서 () loseTeam_rank위로 ()',
                           [u'1순위 내려가',u'한 순위 내려간'],
                           [u'안착하게 되었다.',u'팀의 순위가 하락하였다.']
                           )
        )
        
    #     연승을 하게된 경우
    if parser_DaumKBO.accumulation[0] > 1:
        essentialSentenceGroup.append(
            changeWithParam(u'() winTeam_name은 () () ().',
                            [u'또한',u' '],
                            [u'이번 경기로',u'이번 승리로',u'오늘의 경기로'],
                            [u'winTeam_accumulation연승을 ',u'연속 winTeam_accumulation승을 '],
                            [u'달려나가고 있다.',u'행진하고 있다.',u'이루고 있다.']
                           )
            )
    #     연패를 하게된 경우
    if parser_DaumKBO.accumulation[0] < -1:
        essentialSentenceGroup.append(
            changeWithParam(u'() loseTeam_name은 () loseTeam_accumulation연패를 ()'
                            [u'하지만',u'안타깝게도'],
                            [u'이번 패배로',u'오늘의 패배로']
                            [u'끊어내기 위한 연습이 필요하다.',u'잘라내기위한 노력이 필요하다.']
                           )
        )
    
#     문장뽑기
    sentence1=sentences1Gorup[random.randint(0,len(sentences1)-1)]
    sentence2=sentences2Group[random.randint(0,len(sentences1)-1)]
    
#     문장만들기
    sentence1=sentence1.replace('winTeam_name',Parser_KBO.boxScore['winTeam']['name'])
    sentence1=sentence1.replace('winTeam_win_lose_winCount',str(Parser_DaumKBO.win_lose['winTeam']['win_lose'][0]))
    sentence1=sentence1.replace('winTeam_win_lose_loseCount',str(Parser_DaumKBO.win_lose['winTeam']['win_lose'][2]))
    sentence1=sentence1.replace('winTeam_win_lose_drawCount',str(Parser_DaumKBO.win_lose['winTeam']['win_lose'][1]))
    sentence1=sentence1.replace('winTeam_rank+1',str(Parser_DaumKBO.rank['winTeam'][0]+1))
    sentence1=sentence1.replace('winTeam_rank',str(Parser_DaumKBO.rank['winTeam'][0]))
    sentence1=sentence1.replace('winTeam_accumulation',str(Parser_KBO.accumulation['winTeam']))
    sentence1=sentence1.replace('loseTeam_rank-1',str(Parser_DaumKBO.rank['loseTeam'][0]+1))
    sentence1=sentence1.replace('loseTeam_rank',str(Parser_DaumKBO.rank['loseTeam'][0]))
    sentence1=sentence1.replace('loseTeam_accumulation',str(Parser_KBO.accumulation['loseTeam']))
    sentence1=sentence1.replace('loseTeam_name',Parser_KBO.boxScore['loseTeam']['name'])
    sentence1=sentence1.replace('loseTeam_win_lose_winCount',str(Parser_DaumKBO.win_lose['loseTeam']['win_lose'][0]))
    sentence1=sentence1.replace('loseTeam_win_lose_loseCount',str(Parser_DaumKBO.win_lose['loseTeam']['win_lose'][2]))
    sentence1=sentence1.replace('loseTeam_win_lose_drawCount',str(Parser_DaumKBO.win_lose['loseTeam']['win_lose'][1]))
    sentence1=sentence1.replace('away_win_lose_winCount',str(Parser_DaumKBO.win_lose['away']['win_lose'][0]))
    sentence1=sentence1.replace('away_win_lose_loseCount',str(Parser_DaumKBO.win_lose['away']['win_lose'][2]))
    sentence1=sentence1.replace('away_win_lose_drawCount',str(Parser_DaumKBO.win_lose['away']['win_lose'][1]))
    sentence1=sentence1.replace('home_win_lose_winCount',str(Parser_DaumKBO.win_lose['home']['win_lose'][0]))
    sentence1=sentence1.replace('home_win_lose_loseCount',str(Parser_DaumKBO.win_lose['home']['win_lose'][2]))
    sentence1=sentence1.replace('home_win_lose_drawCount',str(Parser_DaumKBO.win_lose['home']['win_lose'][1]))
    sentence1=sentence1.replace('winTeam_name',Parser_KBO.boxScore['winTeam']['name'])
    sentence1=sentence1.replace('loseTeam_name',Parser_KBO.boxScore['loseTeam']['name'])
    
    Conclusion=sentence1+sentence2
    for line in essentialSentenceGroup:
        Conclusion += line
    
    return Conclusion

In [38]:
def getEmblem(Parser_KBO):
    emblem=getInitialTeamName.get(Parser_KBO.winTeam)
    return emblem

In [49]:
# 본문작성
def writeMain(Parser_KBO,Parser_DaumKBO):
    criticalInning=sorted(Parser_DaumKBO.critical_Inning['home']+Parser_DaumKBO.critical_Inning['away'])
    
#     문장뭉치
#     Main1 : 승부처에 관한 내용
    #------- BEGIN Main1 -------#
    sentence1Group=[
    #         승부처 이닝이 없는 경기에 대한 문장들
        changeWithParam(u''),
        changeWithParam(u'() 이닝없이 경기는 진행되었다.'
                       [u'결정적인']
                       ),
        changeWithParam(u'() 점수차에 영향을 준 이닝은 없었다.',
                       [u'결정적으로',u'크게']
                       ),
        changeWithParam(u'결정적이 이닝이 없었던 () .',
                       [u'그저그런 경기의 연속이었다.'],
                       [u'물흐르듯 지나가는 이닝이 대부분이었다.']
                       ),
        changeWithParam(u'승부가 어느이닝에서 판가름 났다고 얘기하기는 어려웠다.') 
    ] if len(criticalInnin)==0 else [
    #         승부처 이닝이 있는 경기에 대한 문장들
        changeWithParam(u'()의 ()은 () criticalInning (). '
                        [u'승부',u'대결'],
                        [u'양상',u'흐름',u'분위기'],
                        [u'',u'결국'],
                        [u'갈렸다.',u'나뉘었다.',u'두드러졌다.']
                        ),
            u'승부수는 바로 criticalInning에서 갈라지게 되었다. ',
            u'양팀간의 승부는 criticalInning에서 뚜렷하게 나타났다. ',
            u'두팀의 승부는 결국 criticalInning에서 두드러지게 되었다. '
    ]
    #------- END Main1 -------#
    
#     Main2 : 승부처 이닝에 있었던 경기내용
    #------- BEGIN Main2 -------#
    sentence2Group=[u'situation_winTeam_inning_index_player의 situation_winTeam_inning_index_act']
    
    #------- END Main2 -------#

#     문장뽑기
    sentence1=sentences1Gorup[random.randint(0,len(sentences1)-1)]
    sentence2=sentences2Gorup[random.randint(0,len(sentences2)-1)]
    sentence3=sentences3Gorup[random.randint(0,len(sentences3)-1)]
    
#    문장 만들기
    sentence1=sentence1.replace('criticalInning',', '.join((map(str,criticalInning)))
    sentence2=sentence2.replace('situation_winTeam_inning_index_player',)

    
#     Main3 : 키플레이어 관한 내용
    sentence3Group=[
        
    ]
    

#     Main2 BEGIN
    for inn in critical_Inning:
        Main2=u'inn 이닝에서 '
        Main2=Main2.replace(u'inn',inn[0])
#         open All Of Flow
        data=urllib2.urlopen(url+'#sms11')
        html=BeautifulSoup(data)
        cri=str.format('div.situation div#sms%02d > table'%(int(inn[0])))
#                              first = AWAY WIN
        if inn[1]=='first':
            cri=html.select(cri)[0]
#                              second= HOME WIN
        else:
            cri=html.select(cri)[1]
        
        hitContextList=[u'안타',u'루타',u'홈런']
        tmphitList=[]
        hitList=[]
        for line in cri.text.splitlines():
            line = line.strip()
            if line!='':
                tmphitList.append(line)
        for sen in tmphitList[2:]:
            hitList.append(sen.split(' ',2)[2])
        
        for sen in hitList:
#             print sen
            for context in hitContextList:
#                 print context
                if context in sen:
                    player=unicode(sen.split(' ',1)[0])
                    action=unicode(sen.split(' ',1)[1])
                    ex_sen=u'player선수의 action,'
                    ex_sen=ex_sen.replace(u'player',player) 
                    ex_sen=ex_sen.replace(u'action',action)
                    Main2+=ex_sen
                    break
                            
                    
        Main2+=u'등이 Parser_KBO.winTeam의 승리에 큰 기여를 하였다. '
        Main2=Main2.replace(u'Parser_KBO.winTeam',Parser_KBO.winTeam)
#         Main2 END

#         Main3 BEGIN
    Main3=u''
    year=Parser_KBO.date[:4]
    month=Parser_KBO.date[4:6]
    day=Parser_KBO.date[6:8]
    url='http://score.sports.media.daum.net/schedule/baseball/kbo/Main.daum?game_year=%s&game_month=%s'%(year,month)
    page=urllib2.urlopen(url)
    html=BeautifulSoup(page)
    
    s=unicode(html)
    date_index=s.find(str.format('<td class="time_date" rowspan="5">%d<span class="txt_day">')%(int(day)))
    s=s[date_index:]
    
    s=s[s.find(Parser_KBO.a_team):]
    serial_find_url='http://sports.media.daum.net/sports/gamecenter/'
    serial_start_index=s.find(serial_find_url)+len(serial_find_url)
    serial=s[serial_start_index:serial_start_index+8]
    
    url=str.format('http://m.sports.media.daum.net/m/sports/pack/3min/%s?lineup')%(serial)
    
    if os.name=='nt':
        driver=webdriver.PhantomJS(executable_path='./phantomjs.exe')
    else:
        driver=webdriver.PhantomJS(executable_path='./phantomjs')
    driver.get(url)
    html=driver.page_source
#         PhantomJS 드라이버 닫기
    driver.close()
#         Window에서는 PhantomJS프로세스가 남아있으므로 강제종료
    if os.name=='nt':
        os.system('taskkill /f /im phantomjs.exe')
    soup=BeautifulSoup(html)
    
    keyPlayerInfo=[]
    for team,css_select in zip((u'AWAY',u'HOME'),('td.position.away.key-player','td.position.home.key-player')):
        if soup.select_one(css_select)==None:
            continue
        sel=soup.select_one(css_select)
        info={}
        info[u'선수명']=sel.findNextSibling().text
        info[u'홈어웨이']=team
        
        if team==u'AWAY':
            keyPlayerBattingInfo_toDict=Parser_KBO.a_batRecord[Parser_KBO.a_batRecord[u'선수명']==info[u'선수명']]
            keyPlayerBattingInfo_toDict=keyPlayerBattingInfo_toDict.to_dict('records')[0]
        else:
            keyPlayerBattingInfo_toDict=Parser_KBO.h_batRecord[Parser_KBO.h_batRecord[u'선수명']==info[u'선수명']]
            keyPlayerBattingInfo_toDict=keyPlayerBattingInfo_toDict.to_dict('records')[0]
        
        info[u'득점']=keyPlayerBattingInfo_toDict[u'득점']
        info[u'타율']=keyPlayerBattingInfo_toDict[u'타율']
        info[u'타점']=keyPlayerBattingInfo_toDict[u'타점']
        info[u'타수']=keyPlayerBattingInfo_toDict[u'타수']
        info[u'안타']=keyPlayerBattingInfo_toDict[u'안타']
        keyPlayerInfo.append(info)
    
    for kPlayer in keyPlayerInfo:
    #     승리한팀에 키플레이어가 존재하는 경우
        if kPlayer[u'선수명'] in Parser_KBO.winTeam_batRecord[u'선수명'].values:
            Main3=(
                u'Parser_KBO.winTeam의 kPlayer 선수가 Movement 를 기록하여 팀의 키플레이어 역할을 해주었다. ',
                u'Parser_KBO.winTeam kPlayer 선수가 Movement 를 기록 핵심타자 역할을 해주었다. ',
                u'kPlayer선수가 Movement를 하며, Parser_KBO.winTeam의 핵심적인 득점원이 되어주었다. '
            )
            Main3=Main3[random.randint(0,len(Main3)-1)]
            Main3=Main3.replace(u'Parser_KBO.winTeam',Parser_KBO.winTeam)
            Main3=Main3.replace(u'kPlayer',kPlayer[u'선수명'])
            Movement=u'Deukjum Tayul Tajum Tasu Anta'
            Movement=Movement.replace(u'Deukjum',u'득점 '+kPlayer[u'득점'])
            Movement=Movement.replace(u'Tayul',u'타율 '+kPlayer[u'타율'])
            Movement=Movement.replace(u'Tajum',u'타점 '+kPlayer[u'타점'])
            Movement=Movement.replace(u'Tasu',u'타수 '+kPlayer[u'타수'])
            Movement=Movement.replace(u'Anta',u'안타 '+kPlayer[u'안타'])
            Main3=Main3.replace(u'Movement',Movement)

    #     패배한팀에 키플레이어가 존재하는 경우
        else:
            Main3=(
                u'',
                u'Parser_KBO.loseTeam의 kPlayer선수가 Movement 를 하였지만 아쉽게도 팀을 승리로 이끌기에는 부족하였다. ',
                u'kPlayer선수 또한 Movement를 하였지만 Parser_KBO.loseTeam의 승리를 돕기에는 부족했다. ',
                u'Parser_KBO.loseTeam의 kPlayer선수가 Movement 를 하며 열심히 방망이질을 했지만 아쉬운 결과였다. '
                )
            Main3=Main3[random.randint(0,len(Main3)-1)]
            Main3=Main3.replace(u'Parser_KBO.loseTeam',Parser_KBO.loseTeam)
            Main3=Main3.replace(u'kPlayer',kPlayer[u'선수명'])
            Movement=u'Deukjum Tayul Tajum Tasu Anta'
            Movement=Movement.replace(u'Deukjum',u'득점 '+kPlayer[u'득점'])
            Movement=Movement.replace(u'Tayul',u'타율 '+kPlayer[u'타율'])
            Movement=Movement.replace(u'Tajum',u'타점 '+kPlayer[u'타점'])
            Movement=Movement.replace(u'Tasu',u'타수 '+kPlayer[u'타수'])
            Movement=Movement.replace(u'Anta',u'안타 '+kPlayer[u'안타'])
            Main3=Main3.replace(u'Movement',Movement)
     
    return Main1+Main2+Main3

In [5]:
startDate='20160824'
endDate='20160824'
urlParserForKBO = UrlParserForKBO(startDate,endDate)

In [12]:
# MySQL conf
con=MySQLdb.connect(host='218.150.181.131',user='root',passwd='1234',db='link10th',charset='utf8', use_unicode=True)

for url in urlParserForKBO.urlList:
    parser_KBO=Parser_KBO(url)
    parser_DaumKBO=Parser_DaumKBO(parser_KBO.boxScore['date'],parser_KBO.boxScore['awayTeam'])
    try:
        date=writeDate(parser_KBO)
        contextClassifier=getClassifier()
        Head=writeHead(parser_KBO,parser_DaumKBO,contextClassifier)
        Intro=writeIntro(parser_KBO,parser_DaumKBO,contextClassifier)
        Main=writeMain(parser_KBO,parser_DaumKBO)
        Conc=writeConc(parser_KBO,parser_DaumKBO)
        emblem=getEmblem(parser_KBO)
    except:
        sys.stderr.write(unicode.format(u'********** 날짜 : %s %s와 %s의 경기 파싱오류 ********** '%(Parser_KBO.date,Parser_KBO.a_team,Parser_KBO.h_team)))
    
    #교정부분
#     Head = spell_checker.check(Head)
#     Intro = spell_checker.check(Intro)
#     Main = spell_checker.check(Main)
#     Conc = spell_checker.check(Conc)
#     Head = Head.checked
#     Intro = Intro.checked
#     Main = Main.checked
#     Conc = Conc.checked
     
    cursor=con.cursor()
    cursor.execute(
    '''
    INSERT INTO Article(date,Head,Intro,Main,url,emblem,winCount,loseCount,drawCount)
    VALUES (\'%s\',\'%s\',\'%s\',\'%s\',\'%s\',\'%s\',\'%s\'),\'%s\'),\'%s\'))
    '''
    %(date,Head,Intro,Main,url,emblem,winCount,loseCount,drawCount)

    )
    # MySQL disconnect
    con.commit()
    cursor.close()
con.close()

C:\Users\GoHyunyoung98\AppData\Local\Enthought\Canopy\User\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
C:\Users\GoHyunyoung98\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\GoHyunyoung98\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\utils\valida